<a href="https://colab.research.google.com/github/GeorgiyAleksanyan/14-332-472-01-ROBOTICS-COMP-VISION-Classify-MNIST-classes-with-ResNet18/blob/main/ga360_RCV_14_332_472_Project_1_Question_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Classify MNIST classes with ResNet18** Fine-tune the ResNet 18 network to classify the MNIST dataset. Report the confusion matrix, the accuracy, the f-score, precision and recall of your classifier. Write a paragraph describing your results and methods.

In [143]:
!pip install torch~=2.1.0 torchvision pytorch-lightning

In [144]:
!pip install validators matplotlib

In [145]:
import torch
torch.__version__

'2.1.0+cu118'

In [146]:
!pip install -U 'torch_xla>=1.13'
from torchvision.models import resnet18
from torch import nn
from torch.utils.data import DataLoader
from tqdm.autonotebook import tqdm
import pytorch_lightning as pl
from sklearn.metrics import classification_report
import torch
from PIL import Image
import torchvision.transforms as transforms
import numpy as np
import json
import requests
import matplotlib.pyplot as plt
import warnings
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from pytorch_lightning.callbacks import ModelCheckpoint

In [147]:
model = resnet18(num_classes=10)

In [148]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [149]:
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

In [150]:
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor

In [151]:
train_ds = MNIST("mnist", train=True, download=True, transform=ToTensor())
test_ds = MNIST("mnist", train=False, download=True, transform=ToTensor())

In [152]:
train_dl = DataLoader(train_ds, batch_size=64, shuffle=True, num_workers = 1)
test_dl = DataLoader(test_ds, batch_size=64, num_workers = 1)

In [153]:
class ResNetMNIST(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.model = resnet18(num_classes=10)
    self.model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    self.loss = nn.CrossEntropyLoss()

  def forward(self, x):
    return self.model(x)

  def training_step(self, batch, batch_no):
    x, y = batch
    logits = self(x)
    loss = self.loss(logits, y)
    return loss

  def configure_optimizers(self):
    return torch.optim.RMSprop(self.parameters(), lr=0.005)

In [154]:
model = ResNetMNIST()

In [155]:
trainer = pl.Trainer(
    accelerator ="cpu",
    #tpu_cores=8,
    max_epochs=1,
    #callbacks=[ModelCheckpoint(save_top_k=1, monitor="val_loss", mode="min")]
    #progress_bar_refresh_rate=20
)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [156]:
trainer.fit(model, train_dl)

INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | ResNet           | 11.2 M
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.701    Total estimated model params size (MB)
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


In [157]:
trainer.save_checkpoint("resnet18_mnist.pt")

In [158]:
def get_prediction(x, model: pl.LightningModule):
  model.freeze() # prepares model for predicting
  probabilities = torch.softmax(model(x), dim=1)
  predicted_class = torch.argmax(probabilities, dim=1)
  return predicted_class, probabilities

In [160]:
inference_model = ResNetMNIST.load_from_checkpoint("resnet18_mnist.pt", map_location="cpu")

In [162]:
true_y, pred_y = [], []
for batch in tqdm(iter(test_dl), total=len(test_dl)):
  x, y = batch
  true_y.extend(y)
true_y, pred_y = [], []
for batch in tqdm(iter(test_dl), total=len(test_dl)):
  x, y = batch
  true_y.extend(y)
  preds, probs = get_prediction(x, inference_model)
  pred_y.extend(preds.cpu())
[ ]
from sklearn.metrics import classification_report
[ ]
print(classification_report(true_y, pred_y, digits=3))

preds, probs = get_prediction(x, inference_model)
pred_y.extend(preds.cpu())

  0%|          | 0/157 [00:00<?, ?it/s]

  0%|          | 0/157 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0      0.962     0.515     0.671       980
           1      0.856     0.278     0.420      1135
           2      0.802     0.753     0.777      1032
           3      0.779     0.467     0.584      1010
           4      0.972     0.672     0.795       982
           5      0.992     0.568     0.723       892
           6      0.985     0.936     0.960       958
           7      0.225     0.987     0.366      1028
           8      0.925     0.631     0.750       974
           9      0.940     0.234     0.375      1009

    accuracy                          0.600     10000
   macro avg      0.844     0.604     0.642     10000
weighted avg      0.839     0.600     0.636     10000



In [ ]:
#print(classification_report(true_y, pred_y, digits=3))
# Create a confusion matrix
conf_matrix = confusion_matrix(true_y, pred_y)

# Calculate accuracy
accuracy = accuracy_score(true_y, pred_y)

# Calculate F1-score
f1 = f1_score(true_y, pred_y, average='weighted')

# Calculate precision
precision = precision_score(true_y, pred_y, average='weighted')

# Calculate recall
recall = recall_score(true_y, pred_y, average='weighted')

print("Confusion Matrix:")
print(conf_matrix)
print(f"Accuracy: {accuracy:.2f}")
print(f"F1-Score: {f1:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")